# Importing Required Package

In [3]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import tkinter as tk
from tkinter import messagebox
import speech_recognition as sr
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json

# Training the Model

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# Example training data (larger dataset)
X_train = [
    # Scam Messages
    "hello, I am the bank manager, your account is about to expire, please share your details",
    "congratulations! you've won a lottery worth $10,000, please call back",
    "urgent! your bank card has been locked, press 1 to verify your details",
    "immediate action required, your bank account has been compromised",
    "you have been selected for a prize of $500, call us now to claim",
    "your social security number is being used in a fraud, contact us immediately",
    "your account has been suspended for suspicious activities, please verify your information",
    "a suspicious transaction was detected in your account, please confirm via this link",
    
    # Normal Messages
    "I am going to the grocery store later, do you need anything?",
    "Hey, would you like to grab lunch sometime this week?",
    "The meeting has been rescheduled for tomorrow at 2 PM.",
    "Don't forget to submit your homework by Friday!",
    "Looking forward to the weekend trip, hope the weather is nice.",
    "Just wanted to check in and see how you’re doing today.",
    "I tried the new coffee place in town, highly recommend it.",
    "Thanks for the help with the project, it was much appreciated.",
    "Did you see the movie last night? It was awesome!",
    "Let’s catch up soon over a cup of coffee."

]

#Train the vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(X_train)

# Step 2: Save the vectorizer
with open("tfidf_vectorizer.pkl", "wb") as file:
    pickle.dump(vectorizer, file)

print("TF-IDF Vectorizer created and saved as tfidf_vectorizer.pkl")


TF-IDF Vectorizer created and saved as tfidf_vectorizer.pkl


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from nltk.stem import PorterStemmer
import numpy as np
import pickle

# training data with balanced dataset
data = {
    "Call": [
        "hello, I am the bank manager, your account is about to expire, please share your details",
        "your bank account has been compromised. Please call back at this number immediately",
        "we noticed suspicious activity on your account. Please verify your details",
        "please share your credit card details for an important verification",
        "this is to inform you that you have won a lottery. To claim it, share your personal details",
        "urgent action is needed! Your ATM card will be blocked if we don't hear from you immediately",
        "your account is under investigation. Call this number to avoid legal action",
        "congratulations! You’ve won $10,000. Call now to claim your prize",
        "we have detected unusual activity in your bank account. Confirm your identity now",
        "hello sir, this is a government officer. You need to share your Social Security Number urgently",
        "you must wire $500 to unlock your bank account immediately",
        "dear customer, your computer is hacked. Pay now to secure your data",
        "we need immediate verification of your bank information to reactivate your account",
        "your internet service will be suspended due to non-payment. Share details to fix this",
        "act now! This is your final notice to claim the lottery amount of $1,000,000",
        "you will be arrested for unpaid taxes. Contact us immediately to resolve this issue",
        "your phone number has been selected for a reward. Confirm your personal information",
        "Dear user, your account has been hacked. Please reset your password by clicking this link.",
        "Congratulations! You've been selected to win a luxury vacation. Contact us now to confirm your details.",
        "Act fast! Your bank account will be frozen unless you verify your identity right away.",
        "This is the IRS. You owe back taxes and need to pay immediately to avoid serious consequences.",
        "We have detected fraudulent transactions on your account. Call us to resolve the issue.",
        "Urgent! You’ve been charged for a suspicious purchase. Verify your details to prevent further charges.",
        "Hello sir, this is a government officer. You need to share your Social Security Number urgently.",
        "You must wire $500 to unlock your bank account immediately.",
        "Dear customer, your computer is hacked. Pay now to secure your data.",
        "We need immediate verification of your bank information to reactivate your account.",
        "Your internet service will be suspended due to non-payment. Share details to fix this.",
        "Urgent! A suspicious login was detected in your account. Change your password now.",
        "You have been pre-approved for a credit card. Share your details to receive your card today.",
        "Alert! Your bank account is about to expire. Verify now to avoid losing access.",
        "Hello, we have a limited time offer for you. Pay now to claim your prize.",
        "Your account has been blocked due to a suspicious activity report. Please call to resolve this.",
        "Your account has been flagged for unusual activity. Call us immediately to lift the restriction.",
        "Warning! Your account has been compromised. Call us now to regain access.",
        "Important! Your PayPal account has been deactivated. Log in immediately to reactivate.",
        "Your bank card details were exposed. Change your PIN to prevent fraudulent activity.",
        "You’ve won a free iPhone! Just pay a small shipping fee to claim your prize.",
        "Immediate action required! Your bank account has been compromised. Verify your details now.",
        "Congratulations! You have won a lottery worth $1,000,000. To claim it, send your details immediately.",
        "Failure to provide your account verification will lead to suspension of your services.",
        "Act now to resolve legal action against you. Call immediately to avoid fines.",
        "this is a normal call from a friend just to check how you are doing",
        "thank you for your services, we have a special offer for you",
        "this is your regular transaction alert for your account balance",
        "hello, just wanted to let you know about our new services",
        "I’m calling to invite you to my birthday party next week",
        "let’s meet up for coffee tomorrow, what’s a good time for you?",
        "this is a follow-up on your order. It will be delivered on time",
        "good morning! Your monthly bill is ready, you can view it online",
        "hello, the meeting is confirmed for 3 PM tomorrow. See you then",
        "we have a job offer for you based on your recent application",
        "thank you for subscribing to our newsletter. Let us know if you need assistance",
        "this is a friendly reminder for your upcoming dentist appointment",
        "hello, just checking in to make sure you received your package",
        "hello, just checking in to make sure you received your package",
        "congratulations! Your loan application has been approved successfully",
        "we value your feedback, would you like to complete a short survey?",
        "your payment was successful. Have a great day!",
        "it’s me! Just wanted to say hi and catch up soon",
        "I just wanted to remind you about the upcoming event",
        "I found an interesting article I think you will enjoy",
        "we are hosting a webinar on digital marketing. Would you be interested?",
        "your order will be shipped on Monday, so expect it in 5-7 business days",
        "thanks for being a loyal customer. Here's a special thank-you offer for you",
        "how about a weekend road trip? Let’s plan it soon!",
        "thanks for your subscription. Check out our latest blog post",
        "don’t forget our next team meeting at 10 AM tomorrow",
        "reminder: you need to renew your membership before the end of the month",
        "I’ve updated the schedule for this month’s activities. Please check it out",
        "we are so excited to share with you the new product line!",
        "let’s meet up tomorrow afternoon for coffee at 2 PM",
        "we’ve received your feedback and it is greatly appreciated. Thank you!",
        "the gift card you requested has been shipped, it will arrive soon",
        "hope you had a great weekend! Just wanted to touch base",
        "a reminder for your appointment next Wednesday at 11 AM",
        "I’m looking forward to meeting you tomorrow for lunch at noon",
        "we appreciate your feedback and are working to implement some improvements",
        "hope you’re doing well! Let’s catch up soon",
        "congratulations on your recent achievement! Here’s a small token of appreciation"
    ],
    "Decision": [
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    ]
}


# Convert to DataFrame
df = pd.DataFrame(data)

print("Dataset created successfully!")
print(df.head())

# Enhanced preprocessing (removing stop words and stemming)
def preprocess_text(text):
    # Removing stop words
    text = ' '.join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])
    # Stemming
    stemmer = PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text.lower().strip()

# Apply the preprocessing function to the 'Call' column
df['Call'] = df['Call'].apply(preprocess_text)

# Extract features and labels
X = df['Call']  # Input text
y = df['Decision']  # Labels (1 for fraud, 0 for normal)

#TF-IDF Vectorization (using unigrams and bigrams)
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Bigrams along with unigrams
X_tfidf = vectorizer.fit_transform(X)

#Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

#Handle Class Imbalance by adding class_weight parameter
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)
print("Logistic Regression model trained successfully!")

#Cross-validation (use fewer splits, e.g., cv=3)
cv_scores = cross_val_score(model, X_tfidf, y, cv=3)
print(f"Cross-validation scores: {cv_scores}")
print(f"Average CV score: {np.mean(cv_scores)}")

# Model Evaluation
y_pred = model.predict(X_test)

# Step 7: Save the Model and Vectorizer
with open("tfidf_vectorizer.pkl", "wb") as vec_file:
    pickle.dump(vectorizer, vec_file)

with open("logistic_model.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

print("TF-IDF Vectorizer and Logistic Regression Model saved successfully!")


Dataset created successfully!
                                                Call  Decision
0  hello, I am the bank manager, your account is ...         1
1  your bank account has been compromised. Please...         1
2  we noticed suspicious activity on your account...         1
3  please share your credit card details for an i...         1
4  this is to inform you that you have won a lott...         1
Logistic Regression model trained successfully!
Cross-validation scores: [0.85185185 0.85185185 0.96153846]
Average CV score: 0.8884140550807218
TF-IDF Vectorizer and Logistic Regression Model saved successfully!


# SMS Module

In [10]:
from twilio.rest import Client

# Twilio account details
account_sid = 'ACb8576aad18ce8133212bcf7663138028'  # user Account SID
auth_token = 'b3225f48303d45fc3d31b1f69b494365'    # user Auth Token
twilio_number = '+14843571232'     # user Twilio phone number

# Recipient's phone number
recipient_number = '+919361030025'  #  recipient's number

# Message content
message_body = "The caller might be a potential Scammer, be carefull"

# Initialize the Twilio client
client = Client(account_sid, auth_token)

# Send the SMS
def sendOutput(msg,from_no,to_no):
    for _ in range(0,3):
        message = client.messages.create(
            body=msg,
            from_=from_no,
            to=to_no
        )
    print(f"Message sent with SID: {message.sid}")


# Speech to Text Conversion

In [12]:
import speech_recognition as sr
from googletrans import Translator
import pickle
import time

# Load existing ML model and vectorizer
with open("tfidf_vectorizer.pkl", "rb") as file:
    vectorizer = pickle.load(file)

with open("logistic_model.pkl", "rb") as file:
    model = pickle.load(file)

# Recognize Tamil speech and convert to Tamil text
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for Tamil speech...")
        audio = recognizer.listen(source)
        try:
            tamil_text = recognizer.recognize_google(audio, language="ta-IN")
            print(f"Transcribed Tamil text: {tamil_text}")
            return tamil_text
        except sr.UnknownValueError:
            print("Sorry, could not understand. Please try again.")
            return None
        except sr.RequestError as e:
            print(f"Error with Speech Recognition: {e}")
            return None

# Translate Tamil text to English text
def translate_to_english(tamil_text):
    translator = Translator()
    translation = translator.translate(tamil_text, src='ta', dest='en')
    print(f"Translated English text: {translation.text}")
    return translation.text

# Classify the message using the existing ML model
def classify_message(english_text):
    processed_text = preprocess_text(english_text)  # Ensure this function is defined properly for text processing
    vectorized_text = vectorizer.transform([processed_text])
    prediction = model.predict(vectorized_text)
    label = "Fraud" if prediction[0] == 1 else "Normal"
    if label == "Fraud":
        sendOutput(message_body,twilio_number,recipient_number)
    print(f"Prediction: {label}")
    return label

# Project UI

In [14]:
from tkinter import *
import os

'''Window and Title initializing'''
window = Tk()
window.title("Scam Call Detection Using Supervised ML")
window.minsize(width=1200, height=520)
window.config(pady=20, padx=20, bg="white")
window.grid_columnconfigure(4,minsize=300,weight=1)

try:
    '''PTU LOGO'''
    image_path = os.path.join(os.getcwd(),"ptu-logo.png")
    image = PhotoImage(file=image_path)
    resized_image = image.subsample(10, 10)
    window.resized_image = resized_image
    image_label = Label(window, image=resized_image, bg="white")
    image_label.grid(row=1, column=0,rowspan=2,columnspan=2)
    
finally:
        
    row_count = 1
    
    '''Center Content'''
    # Add college title
    college_title = Label(
        text="Puducherry Technological University",
        font=("Arial", 20, "bold"),
        bg="white",
    )
    college_title.grid(row=row_count, column=3, columnspan=2,sticky="s")
    row_count += 1
    # Add Department title
    dept_title = Label(
        text="Department of ECE",
        font=("Arial", 17),
        bg="white",
    )
    dept_title.grid(row=row_count, column=3, columnspan=2)
    row_count += 1
    # Add project title
    project_title = Label(
        text="Scam Call Detection Using Supervised ML",
        font=("Arial", 15),
        bg="white",
    )
    project_title.grid(row=row_count, column=3, columnspan=2)
    row_count += 1
    
    '''Project Main Content'''
    # start of the project
    start_title = Label(text="Start Project : ",font=("Arial", 10, "bold"),bg="white")
    start_title.grid(row=4,column=3, sticky="e",pady=(20,10))
    
    def start_project():
        def update_transcription():
            try:
                tamil_text = speech_to_text()  # Get Tamil speech (function defined elsewhere)
                if tamil_text:
                    # Translate Tamil text to English
                    english_text = translate_to_english(tamil_text)
                    
                    # Update the transcription_content label with the new English text
                    transcription_content.config(text=tamil_text)
                    transcription2_content.config(text=english_text)
                    
    
                    # Call the ML classification function with the new English text
                    result = classify_message(english_text)
                    Decision_content.config(text=result)
                
                # Schedule the next update
                window.after(1000, update_transcription)  # Update every 1000 ms (1 second)
    
            except KeyboardInterrupt:
                print("Real-time listening stopped.")
    
        # Start the update loop
        update_transcription()
        
        
    start_button = Button(text="Click",command=start_project)
    start_button.grid(row=row_count,column=4,sticky="w",pady=(20,10))
    row_count += 1
    
    # audio file transcription
    transcription_title = Label(text="Tamil Transcription : ",font=("Arial", 10, "bold"),bg="white")
    transcription_title.grid(row=row_count,column=3,sticky="e",pady=10)
    transcription_content = Label(text="--None--",font=("Arial", 10),bg="white", wraplength=300)
    transcription_content.grid(row=row_count,column=4,columnspan=2,pady=10,sticky="w")
    row_count += 1

    # audio file transcription
    transcription2_title = Label(text="English Translation : ",font=("Arial", 10, "bold"),bg="white")
    transcription2_title.grid(row=row_count,column=3,sticky="e",pady=10)
    transcription2_content = Label(text="--None--",font=("Arial", 10),bg="white", wraplength=300)
    transcription2_content.grid(row=row_count,column=4,columnspan=2,pady=10,sticky="w")
    row_count += 1
    
    # final output
    Decision_title = Label(text="Decision : ",font=("Arial", 10, "bold"),bg="white")
    Decision_title.grid(row=row_count,column=3,sticky="e",pady=10)
    Decision_content = Label(text="--None--",font=("Arial", 10),bg="white")
    Decision_content.grid(row=row_count,column=4,columnspan=2,pady=10,sticky="w")
    
    '''Credits'''
    '''Guide details'''
    guide_row = row_count + 2
    guide_title = Label(text="Guided by:",font=("Arial", 12,"bold"),bg="white")
    guide_title.grid(row=guide_row,column=2)
    guide_row += 1
    professor_title = Label(text="Dr. Sandanlakshmi R",font=("Arial", 12),bg="white")
    professor_title.grid(row=guide_row,column=2)
    guide_row += 1
    professor_role = Label(text="Associate Professor, ECE",font=("Arial", 12),bg="white")
    professor_role.grid(row=guide_row,column=2)
    
    '''Students Details'''
    team_member = ['Shefaoudeen Z - 21EC1096','Gowtham Balaji R - 21EC1076','Shruthi D - 21EC1097','Thejashri R - 21EC1103']
    student_row =  row_count + 1
    studnet_title = Label(text="Team Members:", font=("Arial", 12,"bold"),bg="white")
    studnet_title.grid(row=student_row,column=5)
    student_row += 1
    for member in team_member:
        student_name= Label(
            text=member,
            font=("Arial", 12),
            bg="white",
        )
        student_name.grid(row=student_row, column=5)
        student_row += 1s
    window.mainloop()

Listening for Tamil speech...
Transcribed Tamil text: ஹலோ
Translated English text: Hello
Prediction: Normal
Listening for Tamil speech...
Sorry, could not understand. Please try again.
Listening for Tamil speech...
Transcribed Tamil text: மை நேம் இஸ் பில்லா
Translated English text: My Name Is Billa
Prediction: Normal
Listening for Tamil speech...
Sorry, could not understand. Please try again.
Listening for Tamil speech...
Transcribed Tamil text: சரி கனரா பேங்க்ல இருந்து பேசுகிறோம் பின் நம்பர் சொல்றீங்களா
Translated English text: Well we are talking from Canara Bangla
Prediction: Normal
Listening for Tamil speech...
Transcribed Tamil text: ஹலோ சார் பேங்க்ல இருந்து பேசுறோம் உங்க ஏடிஎம் பின் நம்பர் சொல்லுங்க அண்ட் சீவி நம்பர் சொல்லுங்க
Translated English text: Hello Sir Bank Talking Your ATM Bin Number Say and Seve
Message sent with SID: SMd925d4756c8e33f88474e6a51af4228c
Prediction: Fraud
Listening for Tamil speech...
Transcribed Tamil text: சார் உங்களுக்கு லாட்டரி வந்துருக்கு ஒரு 5 லேக்